In [6]:
# warmup.py
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    print("GPU ready.")


GPU ready.


In [1]:
import os
import numpy as np
from scipy.spatial import distance
import matplotlib.pyplot as plt
import sqlite3
import tensorflow as tf

2025-03-28 03:47:22.346155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743133643.133169     966 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743133643.348870     966 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743133645.302258     966 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743133645.302290     966 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743133645.302292     966 computation_placer.cc:177] computation placer alr

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import mixed_precision

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)


In [3]:
# model = VGG16()
# model = Model(inputs=model.inputs, outputs = model.layers[-2].output)
model = VGG16(weights="imagenet", include_top=False, pooling="avg")
mixed_precision.set_global_policy("mixed_float16")
print(model.summary())

I0000 00:00:1743133683.529499     966 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6687 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
# def extract_features_for_query(query_img_path):
#     img = load_img(query_img_path, target_size=(224, 224))
#     img_array = img_to_array(img).reshape((1, 224, 224, 3))
#     img_array = preprocess_input(img_array)
#     return model.predict(img_array, verbose=0).flatten()


def extract_features_for_query(query_img_path):
    img = load_img(query_img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array).astype("float16")  # Match mixed precision
    return model.predict(img_array, verbose=0).flatten()


# def search_image(query_img_path, top_n=5):
#     query_vector = extract_features_for_query(query_img_path).reshape(1, -1)

#     conn = sqlite3.connect('features.db')
#     c = conn.cursor()
#     c.execute("SELECT image_path, features FROM image_features")

#     results = []
#     for row in c.fetchall():
#         path, features_str = row
#         feature_vec = np.array(list(map(float, features_str.split(',')))).reshape(1, -1)
#         similarity = cosine_similarity(query_vector, feature_vec)[0][0]
#         results.append((path, similarity))

#     conn.close()
#     results.sort(key=lambda x: x[1], reverse=True)
#     return results[:top_n]


def search_image(query_img_path, top_n=5):
    query_vector = extract_features_for_query(query_img_path).reshape(1, -1)

    conn = sqlite3.connect("features.db")
    c = conn.cursor()
    c.execute("SELECT image_path, features FROM image_features")

    results = []
    for row in c.fetchall():
        path, features_str = row
        feature_vec = np.array(list(map(float, features_str.split(",")))).reshape(1, -1)
        similarity = cosine_similarity(query_vector, feature_vec)[0][0]
        results.append((path, similarity))

    conn.close()
    results.sort(key=lambda x: x[1], reverse=True)
    return results[:top_n]

In [5]:
results = search_image("sample_query.jpg", top_n=3)
# for path, sim in results:
#     print(f"{path} - Similarity: {sim:.4f}")
results

I0000 00:00:1743133687.146810    1794 service.cc:152] XLA service 0x7f8bec0048b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743133687.146852    1794 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2025-03-28 03:48:07.330320: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1743133687.648268    1794 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-28 03:48:09.997638: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.39 = (f32[1,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[1,3,224,224]{3,2,1,0} %bitcast.426, f32[64,3,3,3]{3,2,1,0} %bitcast.433, f32[64]{0} %bitcast.435), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$

[]